# Ayudantía 05: Iterables

## Autores: [@mpvalen](https://github.com/mpvalen), [@Baelfire18](https://github.com/Baelfire18) & [@IgnacioPorte](https://github.com/IgnacioPorte)

Puedes evaluar esta ayudantía en [este link](https://docs.google.com/forms/d/e/1FAIpQLSesBxOc3Ux5hR-da2I1dJJHW-ym9Ho5VDVjCiM4nCYPMmm7tQ/viewform?usp=sf_link)

# Iterables e Iteradores


### ¿Qué es iterar? 

Se dice que "iteramos" sobre alguna estructura de datos cuando recorremos los elementos que la contienen. Es fácil ver que podemos iterar sobre listas, tuplas, diccionarios, *sets*, entre otros. Lo que tal vez no sabías es que puedes crear tus propias estructuras iterables 😱!! Para esto es necesario entender la diferencia entre **iterable** e **iterador**.

### Iterable
Es cualquier objeto sobre el cual **se puede iterar**. Por ejemplo, se puede aplicar un *for loop* con ellos.

Los iterables **implementan** el método `__iter__`, es decir, se puede hacer `iter(iterable)` o `iterable.__iter__()`.



In [3]:
lista = ["No se llama", "Vo teni que ser del Huachipato", "La vieja confiable", "¿Quieres ser tu propio jefe?"]
for i in lista:
    print(i)

print()
print("iter de la lista retorna:", iter(lista))

No se llama
Vo teni que ser del Huachipato
La vieja confiable
¿Quieres ser tu propio jefe?

iter de la lista retorna: <list_iterator object at 0x000002831F437B88>


### Iterador
Un iterador es un objeto que **itera sobre un iterable**, y es retornado por el método `__iter__()` de un iterable.

Los iteradores también implementan el método `__next__()`, el cual retorna el siguiente elemento de la estructura **cada vez que se invoca**. 

Cuando no quedan más elementos por recorrer, se levanta la excepción `StopIteration`.

In [4]:
iterador = iter(lista)

# Notamos que iter(iterador) retorna al mismo iterador
print(f"Soy un {iterador} y mi iter es {iter(iterador)}") 
print(next(iterador))
for i in iterador:    # Se puede iterar sobre ellos, pero...
    print(i)

Soy un <list_iterator object at 0x000002831F443608> y mi iter es <list_iterator object at 0x000002831F443608>
No se llama
Vo teni que ser del Huachipato
La vieja confiable
¿Quieres ser tu propio jefe?


In [12]:
iterador = iter(lista)

for i in iterador:  # Una vez que se recorre completo, no se 'reinicia' como con los iterables.
    print(i)

# Ahora tirará error, pues a pasado el límite
print(next(iterador))

No se llama
Vo teni que ser del Huachipato
La vieja confiable
¿Quieres ser tu propio jefe?


StopIteration: 

**Entonces, ¿cúal es la difencia entre un iterable y un iterador?**

Una iterable es algo recorrible y un iterador permite recorrerlo

**¿Por qué podemos recorrer varias veces un iterable, pero en el caso de un iterador no?**

Para responder a esa pregunta, necesitamos conocer la estructura de un iterable.

A continuación veremos el esqueleto básico de una estructura iterable, creando nuestra propia versión de `range()` (sí, es un iterable).

Primero, definimos nuestro propia clase iterable de rango, con sus atributos y el método `__iter__()`:

In [5]:
class MiIterableRango:
    
    def __init__(self, inicio, fin):
        self.inicio = inicio
        self.actual = inicio
        self.fin = fin
    
    def __iter__(self):
        return MiIterador(self)


Luego definimos nuestro iterador, que debe definir los métodos `__iter__()` y `__next__()`

In [6]:
class MiIterador:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.iterable.actual == self.iterable.fin:
            # Así es como se levanta una excepción del tipo StopIteration
            # con el mensaje "Llegamos al final".
            self.iterable.actual = self.iterable.inicio
            raise StopIteration("We are in the End Game now")
        else:           
            valor = self.iterable.actual
            self.iterable.actual += 1
            return valor

Una vez definidas estas clases, probamos instanciar nuestro rango personalizado:

In [7]:
rango_pulento = MiIterableRango(-3, 3)
for i in rango_pulento:
    print(i)

-3
-2
-1
0
1
2


Notamos también que se cumplen las mismas propiedades de iteradores para el método `iter()` de nuestra clase:

In [8]:
iterador = iter(rango_pulento)
print(next(iterador))
print(next(iterador))

-3
-2


**Logramos definir nuestra propia clase de rango!! 🎉🎉**

Finalmente, llegamos a una idea un poco compleja:

Los *for loops* llaman al método `__iter__()` de la estructura iterable, y para obtener el siguiente valor en la iteración utilizan el método `__next__()` de la clase es retornada por `__iter__()`.

## Generadores

Son un caso especial de los **iteradores**: nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura especial. En pocas palabras, los generadores simplifican la necesidad de crear clases iterables, pero no son tan personalizables como estos últimos.

Para obtener un generador, se puede usar un formato similar a las **listas por comprensión**:

In [9]:
lista_comprension = [i for i in range(5)]
print(lista_comprension)
print(f"Estamos trabajando con {type(lista_comprension)}")

[0, 1, 2, 3, 4]
Estamos trabajando con <class 'list'>


Si reemplazamos los `[]` con `()`, notaremos que hemos creado un generador:

In [10]:
generador = (i for i in range(5))
print(generador)
print(f"Estamos trabajando con {type(generador)}")

<generator object <genexpr> at 0x000002831F42B948>
Estamos trabajando con <class 'generator'>


Del mismo modo que los iteradores, se pueden recorrer usando `for`, y tampoco se 'reinician' como ocurre con los iterables.

In [11]:
for i in generador:
    print(i)

print(next(generador))

0
1
2
3
4


StopIteration: 

## Funciones Generadoras
Es posible crear funciones que funcionen como generadores en Python, a través de la sentencia `yield`. 

`yield` es similar a `return` en retornar un valor indicado, pero en el momento en que la función se vuelve a llamar, se continúa justo donde se había quedado.

Veamos un ejemplo de función generadora.
Consideremos la siguiente serie alternante:
$$ \sum_{n=1}^{m} n(-1)^{n-1}$$

De forma mas simple, lo anterior es equivalente a: 1 - 2 + 3 - 4 + 5 - ...
Hagamos una función generadora usando lo anterior.

In [5]:
def alternante(m):
    pass
generador_alternante = alternante(15)
print(type(generador_alternante))

<class 'NoneType'>


In [43]:
for i in generador_alternante:
    print(i)

1
-1
2
-2
3
-3
4
-4
5
-5
6
-6
7
-7


## Enviar valores a funciones generadoras
Es posible interactuar con funciones generadoras y enviarle datos usando el método `send()`.

Este método envía datos a la expresión `yield` de la función, por lo que al hacer algo como `v = yield valor` el valor enviado con `send()` se guardará en la variable `v`


In [83]:
def funcion_generadora_send():
    contador = 0
    while True:
        recibido = yield contador
        print(f'Recibimos {recibido}')
        if not isinstance(recibido, int):
            print(f'{recibido} no es un entero :c')
            recibido = 0
        contador += recibido
        print(f'Sumamos {recibido} al contador')

In [88]:
generador_send = funcion_generadora_send()

Para poder enviarle algo a la función, es necesario antes llamar al menos una vez a su método `__next__()`, de modo que se alcance la línea que contiene a la expresión `yield`. 

In [89]:
next(generador_send)
generador_send.send(2)
generador_send.send('peko')

Recibimos 2
Sumamos 2 al contador
Recibimos peko
peko no es un entero :c
Sumamos 0 al contador


2

No nos adentraremos mucho en las aplicaciones de este método, pero de todos modos es importante que lo conozcan.

# Funciones built-in



- Hay una herramienta que quizás no muchos no conozcan aún, pero ciertamente resulta útil a la hora de programar. Estamos hablando de las funciones built-in. Lo que las caracteriza es el hecho de que en solo una línea, se puede definir lo que en otras circunstacias tomarían bastantes lineas.


- Hacer uso de este método de programación no solo es eficiente respecto a la longitud de nuestro código, si no que también es *ordenado*.

    Entonces, imaginemos que en **una tarea**, nuestro código requiere un reducir la cantidad de líneas. Podemos usar estas funciones para ahorrarnos un descuento gigante!
**Tip**




Veamos, *qué tipos repasaremos y como funcionan:*

## 1. Funciones Lambda
Las diferencias que tenemos entre una función normal y una lambda, son las siguientes:

- Retornan algo
- Se defininen como funciones anónimas, de manera que no reciben un nombre y tampoco se hace uso de algún nombre para definirlas o llamarlas, si no que se definen a medida que las vayamos a necesitar.

In [5]:
def hipotenusa(a, b): # Función tradicional
    a **=2
    b **= 2
    return (a + b)**(1/2)

print(hipotenusa(3, 4))

5.0


De este modo, tuvimos que definir una función, para posteriormente
necesitar guardar el resultado de interés en una variable.
Lineas en total: 4

In [6]:
#En comparación, tenemos nuestra función hecha mediante el método Lambda
hipotenusa_de_pana = lambda x, y: (x**2 + y**2)**(1/2) # Esta funcion podría haber recibido más argumentos

print(hipotenusa_de_pana(3, 4))

5.0


Como podemos ver, solo tuvimos que usar 1 línea de código, y además
queda ordenado y explícito.

### 2. Función map
- El método map nos permite usar una función lambda que hayamos creado y usarla en uno o más iterables (listas, sets, etc).
- Es importante saber que este método nos entregará un generador, razón por la que también podremos iterar sobre él.
- Veamos a continuación nuestros nuevos alcances...

In [25]:
'''
Creamos una función que podamos replicar mediante lambda y map.
En este caso, lo que haremos será juntar dos listas
'''
nombres = ["María Pía", "Ignacio", "Jose Antonio"]
apellidos = ["Valen", "Porte", "Castro"]

#caso de función normal
def como_te_llamas(nombres, apellidos):
    lista_todo = list()
    for nombre, apellido in zip(nombres, apellidos) :
        persona = " ".join([nombre, apellido])
        lista_todo.append(persona)
    return(lista_todo)

print(como_te_llamas(nombres, apellidos))

#Pero ahora procedemos a definir nuestra función en tan solo una línea!!
nombres_apellidos = map(lambda nombre, apellido: nombre + " " + apellido, nombres, apellidos)
lista_nombres_apellidos = list(nombres_apellidos)
print(lista_nombres_apellidos)
#Tuvimos que poner todo en lista, dado que 

['María Pía Valen', 'Ignacio Porte', 'Jose Antonio Castro']
['María Pía Valen', 'Ignacio Porte', 'Jose Antonio Castro']


Nuevamente, solo usamos una línea. Requiere algo de práctica, es muy útil!

Un buen ejemplo sería a la hora de hacer bases de datos o
trabajar con archivos CSV

### 3. Función filter

Ahora, procederemos a hacer uso de la función filter, mediante ella podemos definir un criterio del tipo *"esto sí", "esto no"*, para categorizar elementos o recuperar de algún iterable los elementos que nos sean útiles o de interés!

In [28]:
def nombres_con_letra_o(iterable):
    lista_auxiliar = []
    for elemento in iterable:
        if "o" in elemento:
            lista_auxiliar.append(elemento)
    return lista_auxiliar

print(nombres_con_letra_o(lista_nombres_apellidos))

['Ignacio Porte', 'Jose Antonio Castro']


In [30]:
nombres_con_letra_o = filter(lambda x: "o" in x, lista_nombres_apellidos)
print(list(nombres_con_letra_o))

['Ignacio Porte', 'Jose Antonio Castro']


### 4. Función reduce
- Esta función **no es built-in**, debes importarla desde functools
- Este caso en particular, corresponde a aplicar un resultado obtenido mediante una función, al elemento siguiente usando la misma función. De manera que se va "guardando" un resultado parcial.
- En términos prácticos, corresponde a aplicar una función sobre una función, sobre una función....

In [31]:
'''
Si observamos en detalle, nuestra operacion va juntando de manera secuencial
las listas, de forma que el resultado final, corresponde a una lista
con todos los elementos que se van iterando

Para este caso en particular, se va iterando de dos en dos
'''
from functools import reduce

lista_con_listas = [[1, 2], [3, 4], [5, 6], [7, 8, 9]]
lista_aplanada = reduce(lambda x, y: x + y, lista_con_listas)
lista_aplanada

[1, 2, 3, 4, 5, 6, 7, 8, 9]

## **Actividad!! 🎉🎉**

¡Han atacado nuevamente el Aeropuerto del DCC! Esta vez unos *ciber-terroristas*, en un intento de causar caos decidieron encriptar el valor de los vuelos.

Tu misión será restaurar los precios originales, después encontrar los vuelos cuyo valor sea mayor a $27 y finalmente encontrar el promedio de los precios de los vuelos. Te han entregado la clase `Lugar` con sus atributos listos para ayudarte a resolver este problema.

El Aeropuerto del DCC sospecha que lo han atacado mediante los *loops for* y *while*, por lo que te han **prohibido usarlos en tu código**

In [14]:
from functools import reduce

encriptado = {"ab": "0", "cd": "1", "ef": "2","gh": "3","ij": "4","kl": "5","mn": "6","op": "7","qr": "8","st": "9"}
class Lugar:
    def __init__(self, nombre, precio):
        self.nombre = nombre
        self.precio = precio
    def desencriptar(self):
        self.precio = int(encriptado[self.precio[0:2]] + encriptado[self.precio[2:4]])
        return self

datos = [['Berlín', "ghab"], ["Cairo", "ghop"], ['Buenos Aires', "cdmn"],
         ['Los Ángeles', "efqr"], ["Tokyo", "efst"], ['New York', "efkl"],
         ['London', "efab"], ['Beijing', "ghab"]]


En primer lugar, nos entregan datos como lista, y debemos convertirlos a objetos.

In [1]:
# ¿Qué función nos puede servir?


Ahora debemos desencriptar los precios usando la función desencriptar

In [16]:
# ¿Qué función nos puede servir?

Luego, debemos obtener solo los nombres de los lugares cuyo precio sea mayor a $27

In [2]:
# ¿Qué función nos puede servir?


Finalmente, nos piden calcular el promedio del precio de los vuelos

In [3]:
# ¿Qué función nos puede servir?